In [29]:
from random import random
from math import sqrt, e
from scipy import stats

---
# Ejercicio 5
Considere una sucesión de números aleatorios $\{U_i\}_i$ y sea $M$ el primer $n$ tal que la variable $U_n$ es menor que su variable predecesora. Es decir,
$$
M = n \quad \text{tal que} \quad U_1 \leq U_2 \leq \ldots \leq U_{n-1} \quad \text{y } U_n < U_{n-1} 
$$

a) Justifique que $P(M > n) = \frac{1}{n!}, \quad n \geqslant 0$

Queremos hallar $P(M > n)$, probabilidad de que $M$, el primer $n$ tal que la variable $U_n$ es menor que su variable predecesora, sea *mayor estricto* que el índice de esa variable.

Tenemos que $\{U_i\}i$ es una sucesión de números aleatorios.

La sucesión va arrojando números aleatorios con distribución uniforme en el $(0,1)$, tal que se tienen:
$$
U_1, U_2, U_3, U_4, \ldots
$$

Supongamos por un momento que se almacenan en una lista tal que:
$$
[U_1, U_2, U_3, U_4, \ldots]
$$

Además supongamos que se generaron *n números aleatorios*:
$$
[U_1, U_2, U_3, U_4, \ldots, U_n]
$$

Ahora, podríamos a todos esos elementos permutarlos de *n formas distintas* con $n!$

Luego $P(M > n)$ significa que de esa lista de n permutaciones tenemos la combinación tal que:
$$
\text{Los } U_i \text{ están ordenados de menor a mayor} \wedge U_n < U_{n-1}
$$

Es decir 1 de las tantas combinaciones.

Luego esa combinación particular es 1 en $n!$

Luego $P(M > n ) = \frac{1}{n!}$

---
b) Utilice la identidad
$$
E[M] = \sum_{n=0}^\infty P(M > n)
$$

Para mostrar que $\displaystyle E[M] = e$

Tenemos por *Taylor* que:
$$
 e^x = \sum_{n=0}^\infty \frac{x^n}{n!}
$$


Supongamos que $x = 1 \Longrightarrow$
$$
 e = \sum_{n=0}^\infty \frac{1^n}{n!} = \sum_{n=0}^\infty \frac{1}{n!} \underset{\text{Ej (a)}}{=} \sum_{n=0}^\infty P(M>n) \underset{Identidad}{=} E[M]
$$

Luego $$\boxed{\displaystyle E[M] = e}$$

---
c) Utilice el resultado del item anterior para dar un estimador de $E[M]$, calcule el valor de su varianza
muestral. Mediante una simulación estime el valor de $e$ deteniéndose cuando la varianza muestral sea
menor que $0,01$.

In [2]:
def M() -> int:
    """
    Variable aleatoria: mínimo n tal que
    U_n-1 > U_n AND U_1 <= U_2 <= ... <= U_n-1

    Returns:
        int: Menor n tal que se cumple lo de antes
    """
    n = 2
    #Caso Base
    U_prev = random()
    U = random()
    if U < U_prev: # En caso de que U_2 < U_1 devuelvo 2
        return n

    while U_prev < U: # General
        n += 1
        U_prev = U
        U = random()        
    return n

In [24]:
def hope_estimator(precision:float) -> tuple[float, int]:
    """
    Estimación de E[M]

    Args:
        precision (float): 0.01 según el ejercicio

    Returns:
        tuple(float, float): Estimación, #iteraciones para calcular la estimación
    """
    i, mean, Scuad = 1, M(), 0
    while not(i > 100 and sqrt(Scuad/i) < precision):
        i += 1
        U_i = M()
        prev_mean = mean
        mean = prev_mean + (U_i - prev_mean) / i
        Scuad = Scuad * (1 - 1 / (i - 1)) + i * (mean - prev_mean) ** 2
    return mean, i

In [53]:
results = hope_estimator(precision=0.01)
print(f"🎯 VALOR EXACTO: e = {e}")
print(f"🤏🏽   ESTIMACIÓN: e ≈ {results[0]}")
print(f"📝 #ITERACIONES: N = {results[1]}")
print(f"🤬    ERROR ABS: Δ = {abs(e-results[0]):.15f}")

🎯 VALOR EXACTO: e = 2.718281828459045
🤏🏽   ESTIMACIÓN: e ≈ 2.7315255108597913
📝 #ITERACIONES: N = 7781
🤬    ERROR ABS: Δ = 0.013243682400746


---
d) Dé una estimación de $e$ mediante un intervalo de ancho menor que $0,1$ y con una confianza del $95%$

In [31]:
def number_estimation_w_CI(precision: float) -> tuple[float, float]:
    """
    Estimación Numérica
    - Calcula una estimación del número e
    Utilizando un intervalo de confianza del 95%

    Args:
        Nsim (int): Número de simulaciones

    Returns:
        tuple(float, float): Estimación numérica de e, #iteraciones para calcular la estimación
    """
    mean, Scuad = M(), 0
    i = 1
    while not(i > 100 and sqrt(Scuad / i) < precision):
        i += 1
        X = M()
        prev_mean = mean
        mean = prev_mean + (X - prev_mean) / i
        Scuad = Scuad * (1 - 1 / (i - 1)) + i * (mean - prev_mean) ** 2
    return mean, i

In [54]:
L = 0.1
confiance = 0.95
alpha = 1 - confiance
z_alpha_2 = abs(stats.norm.ppf(alpha/2))
precision = L / (2 * z_alpha_2)
results = number_estimation_w_CI(precision=float(precision))
print(f"🎯 VALOR EXACTO: e = {e}")
print(f"🤏🏽   ESTIMACIÓN: e ≈ {results[0]}")
print(f"📝 #ITERACIONES: N = {results[1]}")
print(f"🤬    ERROR ABS: Δ = {abs(e-results[0]):.15f}")

🎯 VALOR EXACTO: e = 2.718281828459045
🤏🏽   ESTIMACIÓN: e ≈ 2.743922204213945
📝 #ITERACIONES: N = 1234
🤬    ERROR ABS: Δ = 0.025640375754900


# Comparaciones de ambos métodos

| Método de Estimación                        | 🎯 Valor Exacto (e) | 🤏🏽 Estimación       | 📝 Iteraciones (N) | 🤬 Error Absoluto (Δ) |
|--------------------------------------------|---------------------|----------------------|---------------------|------------------------|
| Estimación directa de E[M]                 | 2.718281828459045   | ≈ 2.7315255108597913 | 7781                | 0.013243682400746      |
| Intervalo de confianza del 95%             | 2.718281828459045   | ≈ 2.743922204213945  | 1234                | 0.025640375754900      |


Se puede ver que con muchísimas menos iteraciones con un intervalo de confianza y con la precisión dada podemos
aproximarnos mucho y cometer un error dentro de todo bajo a comparación del método por estimación directa de $E[M]$